## Vector Search with IRIS SQL
This tutorial covers how to use IRIS as a vector database. 

For this tutorial, we will use a dataset of 2.2k online reviews of scotch (
dataset from https://www.kaggle.com/datasets/koki25ando/22000-scotch-whisky-reviews) . With our latest vector database functionality, we can leverage the latest embedding models to run semantic search on the online reviews of scotch whiskeys. In addition, we'll be able to apply filters on columns with structured data. For example, we will be able to search for whiskeys that are priced under $100, and are 'earthy, smooth, and easy to drink'. Let's find our perfect whiskey!

In [1]:
import os, pandas as pd
from sentence_transformers import SentenceTransformer
from sqlalchemy import create_engine, text

/opt/anaconda3/envs/iris-vector-search/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
username = 'demo'
password = 'demo'
hostname = os.getenv('IRIS_HOSTNAME', 'localhost')
port = '1972' 
namespace = 'USER'
CONNECTION_STRING = f"iris://{username}:{password}@{hostname}:{port}/{namespace}"

In [3]:
engine = create_engine(CONNECTION_STRING)

In [4]:
# Load the CSV file
df = pd.read_csv('../data/scotch_review.csv')

In [5]:
df.head()

,Unnamed: 0,name,category,review.point,price,currency,description
0,1,"Johnnie Walker Blue Label, 40%",Blended Scotch Whisky,97,225,$,"Magnificently powerful and intense. Caramels, ..."
1,2,"Black Bowmore, 1964 vintage, 42 year old, 40.5%",Single Malt Scotch,97,4500.00,$,What impresses me most is how this whisky evol...
2,3,"Bowmore 46 year old (distilled 1964), 42.9%",Single Malt Scotch,97,13500.00,$,There have been some legendary Bowmores from t...
3,4,"Compass Box The General, 53.4%",Blended Malt Scotch Whisky,96,325,$,With a name inspired by a 1926 Buster Keaton m...
4,5,"Chivas Regal Ultis, 40%",Blended Malt Scotch Whisky,96,160,$,"Captivating, enticing, and wonderfully charmin..."


In [6]:
# Clean data
# Remove the specified columns
df.drop(['currency'], axis=1, inplace=True)

# Drop the first column
df.drop(columns=df.columns[0], inplace=True)

# Remove rows without a price
df.dropna(subset=['price'], inplace=True)

# Ensure values in 'price' are numbers
df = df[pd.to_numeric(df['price'], errors='coerce').notna()]

# Replace NaN values in other columns with an empty string
df.fillna('', inplace=True)

In [7]:
df.head()

,name,category,review.point,price,description
0,"Johnnie Walker Blue Label, 40%",Blended Scotch Whisky,97,225,"Magnificently powerful and intense. Caramels, ..."
1,"Black Bowmore, 1964 vintage, 42 year old, 40.5%",Single Malt Scotch,97,4500.00,What impresses me most is how this whisky evol...
2,"Bowmore 46 year old (distilled 1964), 42.9%",Single Malt Scotch,97,13500.00,There have been some legendary Bowmores from t...
3,"Compass Box The General, 53.4%",Blended Malt Scotch Whisky,96,325,With a name inspired by a 1926 Buster Keaton m...
4,"Chivas Regal Ultis, 40%",Blended Malt Scotch Whisky,96,160,"Captivating, enticing, and wonderfully charmin..."


Now, InterSystems IRIS supports vectors as a datatype in tables! Here, we create a table with a few different columns. The last column, 'description_vector', will be used to store vectors that are generated by passing the 'description' of a review through an embedding model.

In [8]:

with engine.connect() as conn:
    with conn.begin():# Load 
        sql = f"""
                CREATE TABLE IF NOT EXISTS scotch_reviews (
        name VARCHAR(255),
        category VARCHAR(255),
        review_point INT,
        price DOUBLE,
        description VARCHAR(2000),
        description_vector VECTOR(DOUBLE, 384)
        )
                """
        result = conn.execute(text(sql))

In [9]:
# Load a pre-trained sentence transformer model. This model's output vectors are of size 384
model = SentenceTransformer('all-MiniLM-L6-v2') 

In [10]:

# Generate embeddings for all descriptions at once. Batch processing makes it faster
embeddings = model.encode(df['description'].tolist(), normalize_embeddings=True)

# Add the embeddings to the DataFrame
df['description_vector'] = embeddings.tolist()


In [11]:
df.head()

,name,category,review.point,price,description,description_vector
0,"Johnnie Walker Blue Label, 40%",Blended Scotch Whisky,97,225,"Magnificently powerful and intense. Caramels, ...","[-0.010494445450603962, 0.014728965237736702, ..."
1,"Black Bowmore, 1964 vintage, 42 year old, 40.5%",Single Malt Scotch,97,4500.00,What impresses me most is how this whisky evol...,"[0.02318125031888485, -0.05123035982251167, 0...."
2,"Bowmore 46 year old (distilled 1964), 42.9%",Single Malt Scotch,97,13500.00,There have been some legendary Bowmores from t...,"[0.04333321005105972, -0.017066635191440582, -..."
3,"Compass Box The General, 53.4%",Blended Malt Scotch Whisky,96,325,With a name inspired by a 1926 Buster Keaton m...,"[-0.07594005018472672, -0.03676239028573036, 0..."
4,"Chivas Regal Ultis, 40%",Blended Malt Scotch Whisky,96,160,"Captivating, enticing, and wonderfully charmin...","[-0.012818857096135616, -0.09769789129495621, ..."


In [12]:
with engine.connect() as conn:
    with conn.begin():
        for index, row in df.iterrows():
            sql = text("""
                INSERT INTO scotch_reviews 
                (name, category, review_point, price, description, description_vector) 
                VALUES (:name, :category, :review_point, :price, :description, TO_VECTOR(:description_vector))
            """)
            conn.execute(sql, {
                'name': row['name'], 
                'category': row['category'], 
                'review_point': row['review.point'], 
                'price': row['price'], 
                'description': row['description'], 
                'description_vector': str(row['description_vector'])
            })


Let's look for a scotch that costs less than $100, and has an earthy and creamy taste.

In [13]:
description_search = "earthy and creamy taste"
search_vector = model.encode(description_search, normalize_embeddings=True).tolist() # Convert search phrase into a vector

In [14]:
with engine.connect() as conn:
    with conn.begin():
        sql = text("""
            SELECT DISTINCT TOP 3 * FROM scotch_reviews 
            WHERE price < 100 
            ORDER BY VECTOR_DOT_PRODUCT(description_vector, TO_VECTOR(:search_vector)) DESC
        """)

        results = conn.execute(sql, {'search_vector': str(search_vector)}).fetchall()


In [15]:
print(results)

[('SIGNATORY (DISTILLED AT BOWMORE), 16 YEAR OLD, 1988 VINTAGE, CASK #42508, 46%', 'SINGLE MALT SCOTCH', 87, 60.0, 'MEDIUM-BODIED AND NICELY TEXTURED. GOOD BALANCE OF FLAVORS -- AND WELL-INTEGRATED, TOO -- WITH LOVELY SWEET NOTES (CEREAL GRAIN, COOKIE DOUGH, CARAME ... (48 characters truncated) ...  FISHNETS, AND BRINE THAT IS COMPLEMENTARY, BUT NOT AGGRESSIVE, WITH A SUGGESTION OF LAVENDER AND TANGERINE. BALANCED FINISH. (332 BOTTLES PRODUCED.)', '-.048620376735925674438,-.082065843045711517333,.039660684764385223388,-.018970852717757225036,-.017485298216342926026,.042453121393918991088,.046325 ... (8848 characters truncated) ... 064819,-.0038620312698185443878,-.022344633936882019042,.052769336849451065063,-.061306387186050415039,.048756919801235198974,-.063436612486839294433'), ('SHIELDAIG 12 YEAR OLD, 40%', 'BLENDED SCOTCH WHISKY', 85, 31.0, 'THIS IS A SHARP DRESSER, WITH A FIRM, SOLID MOUTHFEEL AND AN ALTOGETHER FINER AND MORE FOCUSED TASTE THAN SHIELDAIG CLASSIC (SEE\r\nBELOW). I

In [16]:
results_df = pd.DataFrame(results, columns=df.columns).iloc[:, :-1] # Remove vector
pd.set_option('display.max_colwidth', None)  # Easier to read description
results_df.head()

,name,category,review.point,price,description
0,"SIGNATORY (DISTILLED AT BOWMORE), 16 YEAR OLD, 1988 VINTAGE, CASK #42508, 46%",SINGLE MALT SCOTCH,87,60.0,"MEDIUM-BODIED AND NICELY TEXTURED. GOOD BALANCE OF FLAVORS -- AND WELL-INTEGRATED, TOO -- WITH LOVELY SWEET NOTES (CEREAL GRAIN, COOKIE DOUGH, CARAMEL, AND VANILLA CREAM), YOUNG HEATHERY PEAT, TAR, FISHNETS, AND BRINE THAT IS COMPLEMENTARY, BUT NOT AGGRESSIVE, WITH A SUGGESTION OF LAVENDER AND TANGERINE. BALANCED FINISH. (332 BOTTLES PRODUCED.)"
1,"SHIELDAIG 12 YEAR OLD, 40%",BLENDED SCOTCH WHISKY,85,31.0,"THIS IS A SHARP DRESSER, WITH A FIRM, SOLID MOUTHFEEL AND AN ALTOGETHER FINER AND MORE FOCUSED TASTE THAN SHIELDAIG CLASSIC (SEE\r\nBELOW). IT’S NOT COASTAL OR EARTHY PARTICULARLY, EITHER. INSTEAD THE FLAVORS ARE SOFTER AND BUILT AROUND MOCHA, SMOOTH CREAMY TOFFEE, AND SOME SOFT FRUIT, INCLUDING A TOUCH OF OVERRIPE BANANA AND MELON NOTES. THE SAVORINESS THIS TIME COMES FROM A TOUCH OF PEPPER RATHER THAN SALT."
2,"THE ARRAN MALT, SINGLE BOURBON CASK, (CASK#1801), 1996 VINTAGE, 50.5%",SINGLE MALT SCOTCH,86,80.0,"FRESH AND CLEAN, WITH NOTES OF VANILLA, RIPE BARLEY, HONEY, CARAMEL APPLE, AND TOASTED COCONUT. CREAMY AND MOUTH-COATING IN TEXTURE, LEADING TO A PLEASINGLY DRY, SPICY OAK FINISH. VERY DRINKABLE, YET SATISFYING. QUITE NICE. \r\n"


## Vector Index

The latest version of IRIS released to the Early Access Program includes not only bug fixes and performance enhancements, but a new disk-based Approximate Nearest Neighbors index that speeds up vector search for large collections of vectors (typically over 100K).

Since this notebook is working with a dataset much smaller than 100K rows, there won't be a measurable performance benefit, and this is provided as an example you can adapt.

NOTE: Please note this feature is currently targeted at 2025.1. Please join the [Early Access Program](https://live.evaluation.iscinternal.com/download/adminearlyaccess.csp?earlyAccessProgram=Vector_Search) if you'd like to work with a preview kit.

In [ ]:
with engine.connect() as conn:
    with conn.begin():# Load 
        sql = f"""
                CREATE INDEX HNSWIndex 
                ON TABLE scotch_reviews (description_vector)
                AS %SQL.Index.HNSW(M=80, Distance='DotProduct')
                """
        result = conn.execute(text(sql))